In [3]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import minibatch, compounding
import pandas as pd
import json
import re
import os
import random
from datetime import datetime
from pathlib import Path

class SirenTrainer:
    """Classe pour extraire des SIREN et entraîner un modèle spaCy"""
    
    def __init__(self, modele_base="fr_core_news_md", dossier_donnees="donnees_entrainement"):
        self.modele_base = modele_base
        self.dossier_donnees = Path(dossier_donnees)
        self.dossier_donnees.mkdir(exist_ok=True)
        
        # Données d'entraînement de base
        self.train_data_base = [
            ("La société X est identifiée par le SIREN 918090010.", {"entities": [(39, 48, "SIREN")]}),
            ("SIREN : 981086762", {"entities": [(8, 17, "SIREN")]}),
            ("Numéro SIREN 822 210 944 valide.", {"entities": [(13, 24, "SIREN")]}),
            ("Identifiée sous le SIREN 938263548", {"entities": [(25, 34, "SIREN")]}),
            ("Le SIREN est 942558057", {"entities": [(13, 22, "SIREN")]}),
            ("Siège social 10 rue des Crombions 62840 FLEURBAIX 881 109 649 RCS Arras", {"entities": [(50, 59, "SIREN")]}),
            ("RCS Lille 123 456 789", {"entities": [(10, 21, "SIREN")]}),
            ("Immatriculée sous le numéro SIREN 987654321", {"entities": [(34, 43, "SIREN")]}),
            ("SIREN123456789", {"entities": [(5, 14, "SIREN")]}),
            ("Entreprise immatriculée au RCS de Nantes sous le n° 456 789 123", {"entities": [(52, 63, "SIREN")]}),
            ("Numéro d'identification 321654987 selon l'INSEE", {"entities": [(25, 34, "SIREN")]}),
            ("Le numéro est : 789654123", {"entities": [(16, 25, "SIREN")]}),
            ("Numéro de SIREN:123 123 123", {"entities": [(17, 28, "SIREN")]}),
            ("N° SIREN : 741852963", {"entities": [(11, 20, "SIREN")]}),
        ]
        
        # SIREN supplémentaires pour générer plus de données
        self.liste_siren_base = ["123456789", "741852963", "987654321"]
    
    def valider_siren(self, siren_str):
        """Valider un numéro SIREN avec l'algorithme de Luhn"""
        siren = re.sub(r'\D', '', str(siren_str))
        if len(siren) != 9 or not siren.isdigit():
            return False
        
        total = 0
        for i, digit in enumerate(siren):
            n = int(digit)
            if i % 2 == 1:
                n *= 2
                if n > 9:
                    n -= 9
            total += n
        
        return total % 10 == 0
    
    def extraire_siren_du_texte(self, texte):
        """Extraire les SIREN d'un texte avec regex"""
        patterns = [
            r'\b\d{9}\b',
            r'\b\d{3}[\s-]\d{3}[\s-]\d{3}\b',
            r'\b\d{3}\s+\d{3}\s+\d{3}\b'
        ]
        
        numeros_siren = []
        for pattern in patterns:
            for match in re.finditer(pattern, texte):
                siren_propre = re.sub(r'\D', '', match.group())
                if len(siren_propre) == 9:
                    numeros_siren.append(siren_propre)
        
        return list(dict.fromkeys(numeros_siren))
    
    def generer_donnees_entrainement(self, sirens_extraits):
        """Générer des données d'entraînement à partir des SIREN extraits"""
        templates = [
            "La société est immatriculée sous le numéro {siren}",
            "SIREN : {siren}",
            "Le SIREN de l'entreprise est {siren}.",
            "Numéro SIREN {siren} validé",
            "RCS Paris {siren}",
            "Identifiant SIREN: {siren}",
            "N° SIREN : {siren}",
            "Société immatriculée {siren}",
            "SIREN {siren} entreprise",
            "Numéro d'identification {siren}",
            "Enregistrée sous le SIREN {siren}",
            "Code SIREN {siren}",
            "Immatriculation SIREN : {siren}",
            "Le numéro SIREN est {siren}",
            "SIREN de la société : {siren}"
        ]
        
        nouvelles_donnees = []
        
        for siren in sirens_extraits:
            if self.valider_siren(siren):  # Utiliser seulement les SIREN valides
                for template in templates:
                    texte = template.format(siren=siren)
                    
                    # Trouver la position du SIREN dans le texte
                    start = texte.find(siren)
                    end = start + len(siren)
                    
                    if start != -1:
                        nouvelles_donnees.append((texte, {"entities": [(start, end, "SIREN")]}))
        
        return nouvelles_donnees
    
    def traiter_fichier_json(self, chemin_json):
        """Traiter un fichier JSON pour extraire les SIREN"""
        try:
            with open(chemin_json, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Extraire le texte selon la structure du JSON
            texte_complet = ""
            
            if isinstance(data, dict):
                # Parcourir récursivement le JSON pour extraire le texte
                texte_complet = self._extraire_texte_json(data)
            elif isinstance(data, list):
                for item in data:
                    texte_complet += self._extraire_texte_json(item) + " "
            
            # Extraire les SIREN du texte
            sirens_extraits = self.extraire_siren_du_texte(texte_complet)
            
            return sirens_extraits, texte_complet
            
        except Exception as e:
            print(f"Erreur lors du traitement du fichier {chemin_json}: {e}")
            return [], ""
    
    def _extraire_texte_json(self, obj):
        """Extraire récursivement le texte d'un objet JSON"""
        texte = ""
        
        if isinstance(obj, dict):
            for key, value in obj.items():
                texte += self._extraire_texte_json(value) + " "
        elif isinstance(obj, list):
            for item in obj:
                texte += self._extraire_texte_json(item) + " "
        elif isinstance(obj, str):
            texte += obj + " "
        
        return texte
    
    def traiter_dossier_json(self, dossier_json):
        """Traiter tous les fichiers JSON d'un dossier"""
        dossier_path = Path(dossier_json)
        if not dossier_path.exists():
            print(f"Le dossier {dossier_json} n'existe pas.")
            return []
        
        tous_sirens = []
        fichiers_traites = []
        
        for fichier_json in dossier_path.glob("*.json"):
            print(f"Traitement de {fichier_json.name}...")
            sirens, texte = self.traiter_fichier_json(fichier_json)
            
            if sirens:
                tous_sirens.extend(sirens)
                fichiers_traites.append({
                    'fichier': fichier_json.name,
                    'sirens': sirens,
                    'nb_sirens': len(sirens)
                })
        
        print(f"\\nTraitement terminé:")
        print(f"- {len(fichiers_traites)} fichiers traités")
        print(f"- {len(set(tous_sirens))} SIREN uniques extraits")
        
        return list(set(tous_sirens))  # Retourner les SIREN uniques
    
    def creer_dataset_entrainement(self, sirens_extraits=None, nom_fichier="train_siren.spacy"):
        """Créer un dataset d'entraînement complet"""
        
        # Commencer avec les données de base
        train_data = self.train_data_base.copy()
        
        # Ajouter les données générées à partir des SIREN de base
        for siren in self.liste_siren_base:
            train_data.extend(self.generer_donnees_entrainement([siren]))
        
        # Ajouter les données des SIREN extraits
        if sirens_extraits:
            nouvelles_donnees = self.generer_donnees_entrainement(sirens_extraits)
            train_data.extend(nouvelles_donnees)
        
        # Mélanger les données
        random.shuffle(train_data)
        
        # Créer le DocBin
        nlp = spacy.blank("fr")
        doc_bin = DocBin()
        
        donnees_valides = 0
        
        for text, annot in train_data:
            doc = nlp.make_doc(text)
            ents = []
            
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label)
                if span:
                    ents.append(span)
            
            if ents:  # Ajouter seulement si des entités ont été trouvées
                doc.ents = ents
                doc_bin.add(doc)
                donnees_valides += 1
        
        # Sauvegarder
        chemin_sortie = self.dossier_donnees / nom_fichier
        doc_bin.to_disk(chemin_sortie)
        
        print(f"Dataset créé avec {donnees_valides} exemples valides")
        print(f"Sauvegardé dans: {chemin_sortie}")
        
        return str(chemin_sortie)
    
    def entrainer_modele(self, chemin_dataset, nom_modele="modele_siren", n_iter=30):
        """Entraîner un modèle spaCy pour la reconnaissance des SIREN"""
        
        try:
            # Charger le modèle de base
            nlp = spacy.load(self.modele_base)
            print(f"Modèle de base chargé: {self.modele_base}")
        except OSError:
            print(f"Modèle {self.modele_base} non trouvé. Utilisation d'un modèle vide.")
            nlp = spacy.blank("fr")
        
        # Ajouter le composant NER s'il n'existe pas
        if "ner" not in nlp.pipe_names:
            ner = nlp.add_pipe("ner")
        else:
            ner = nlp.get_pipe("ner")
        
        # Ajouter le label SIREN
        ner.add_label("SIREN")
        
        # Charger les données d'entraînement
        doc_bin = DocBin().from_disk(chemin_dataset)
        docs = list(doc_bin.get_docs(nlp.vocab))
        
        # Créer les exemples d'entraînement
        examples = []
        for doc in docs:
            example = Example.from_dict(doc, {"entities": [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]})
            examples.append(example)
        
        print(f"Entraînement avec {len(examples)} exemples...")
        
        # Désactiver les autres pipes pendant l'entraînement
        unaffected_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
        
        with nlp.disable_pipes(*unaffected_pipes):
            # Commencer l'entraînement
            nlp.begin_training()
            
            for iteration in range(n_iter):
                random.shuffle(examples)
                losses = {}
                
                # Entraîner par batches
                batches = minibatch(examples, size=compounding(4.0, 32.0, 1.001))
                for batch in batches:
                    nlp.update(batch, drop=0.5, losses=losses)
                
                if iteration % 5 == 0:
                    print(f"Itération {iteration}: Perte = {losses.get('ner', 0):.3f}")
        
        # Sauvegarder le modèle
        dossier_modele = self.dossier_donnees / nom_modele
        nlp.to_disk(dossier_modele)
        
        print(f"\\nModèle entraîné et sauvegardé dans: {dossier_modele}")
        return str(dossier_modele)
    
    def tester_modele(self, chemin_modele, textes_test=None):
        """Tester le modèle entraîné"""
        try:
            nlp = spacy.load(chemin_modele)
            
            if textes_test is None:
                textes_test = [
                    "La société ACME a le SIREN 123456789",
                    "RCS Paris 987654321",
                    "Numéro SIREN : 741852963",
                    "L'entreprise est immatriculée sous le numéro 456789123"
                ]
            
            print("\\n=== TEST DU MODÈLE ===")
            for texte in textes_test:
                doc = nlp(texte)
                print(f"\\nTexte: {texte}")
                if doc.ents:
                    for ent in doc.ents:
                        if ent.label_ == "SIREN":
                            print(f"  → SIREN détecté: {ent.text} (confiance: {ent._.confidence if hasattr(ent._, 'confidence') else 'N/A'})")
                else:
                    print("  → Aucun SIREN détecté")
                    
        except Exception as e:
            print(f"Erreur lors du test: {e}")

def main():
    """Fonction principale"""
    # Configuration
    trainer = SirenTrainer()
    
    # Étape 1: Traiter les fichiers JSON
    dossier_json = input("Entrez le chemin du dossier contenant les fichiers JSON: ").strip()
    if not dossier_json:
        dossier_json = "fichiers_json"  # Dossier par défaut
    
    print("\\n=== EXTRACTION DES SIREN DEPUIS LES FICHIERS JSON ===")
    sirens_extraits = trainer.traiter_dossier_json(dossier_json)
    
    if sirens_extraits:
        print(f"\\nSIREN extraits: {sirens_extraits}")
        
        # Étape 2: Créer le dataset d'entraînement
        print("\\n=== CRÉATION DU DATASET D'ENTRAÎNEMENT ===")
        chemin_dataset = trainer.creer_dataset_entrainement(sirens_extraits)
        
        # Étape 3: Entraîner le modèle
        print("\\n=== ENTRAÎNEMENT DU MODÈLE ===")
        chemin_modele = trainer.entrainer_modele(chemin_dataset)
        
        # Étape 4: Tester le modèle
        print("\\n=== TEST DU MODÈLE ===")
        trainer.tester_modele(chemin_modele)
        
        print(f"\\n✅ Processus terminé avec succès!")
        print(f"📁 Modèle sauvegardé dans: {chemin_modele}")
        
    else:
        print("Aucun SIREN trouvé dans les fichiers JSON.")
        
        # Créer quand même un dataset avec les données de base
        print("\\nCréation d'un dataset avec les données de base...")
        chemin_dataset = trainer.creer_dataset_entrainement()
        chemin_modele = trainer.entrainer_modele(chemin_dataset)
        trainer.tester_modele(chemin_modele)

if __name__ == "__main__":
    main()


\n=== EXTRACTION DES SIREN DEPUIS LES FICHIERS JSON ===
Traitement de siren_extraits_20250611_113328.json...
Traitement de siren_extraits_20250611_115313.json...
Traitement de siren_extraits_20250611_120916.json...
Traitement de siren_extraits_20250612_091547.json...
Traitement de siren_extraits_20250612_092246.json...
Traitement de siren_extraits_20250612_100830.json...
Traitement de siren_extraits_20250612_105258.json...
\nTraitement terminé:
- 5 fichiers traités
- 468 SIREN uniques extraits
\nSIREN extraits: ['378657746', '901182303', '483445565', '783691728', '182489760', '927682864', '953849312', '953930252', '922239918', '515230613', '820021475', '493669683', '517680401', '818843609', '910337716', '904032646', '883918195', '842571234', '313285181', '929263515', '533675443', '798440798', '513194027', '901603555', '345310965', '504760323', '484551767', '492794490', '929987204', '808775639', '851389429', '844927905', '899243885', '811568120', '452951874', '903375954', '981784564', '